In [1]:
from datetime import datetime

from binance.client import Client
from binance.enums import *

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from datetime import datetime

import numpy as np
import pandas as pd
import json
import time


In [2]:
client = Client('', '')

In [3]:
cmc = CoinMarketCapAPI('')

In [4]:
info = client.get_all_tickers()

In [5]:
quote_ccy = 'USDT'

In [6]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith(quote_ccy)]

In [7]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [8]:
target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 300 and d['rank'] <= 350]

In [9]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [10]:
target_ticker_list

['FETUSDT',
 'MBLUSDT',
 'STPTUSDT',
 'ANTUSDT',
 'SUNUSDT',
 'UTKUSDT',
 'XVSUSDT',
 'ALPHAUSDT',
 'CTKUSDT',
 'REEFUSDT',
 'ALICEUSDT',
 'PERPUSDT',
 'TLMUSDT',
 'BAKEUSDT',
 'POLSUSDT',
 'XVGUSDT',
 'GHSTUSDT',
 'TRIBEUSDT',
 'DARUSDT',
 'RGTUSDT',
 'MOVRUSDT',
 'ACHUSDT']

In [11]:
# target_ticker_list = ['BONDUSDT']

In [12]:
def is_candidate(filtered_klines_info, tolerable_volume): 
    context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
    is_candidate = False
    
    
    trigger_kline = filtered_klines_info[-1]
    
    
    for context_kline in context_klines_info:
        kontext_tol_val = tolerable_volume * 2
        
        volume_gate = float(context_kline[5])  <  kontext_tol_val
        if volume_gate :
            is_candidate = True
        else:
            is_candidate = True
            

    #trigger_gate = float(trigger_kline[5]  > tolerable_volume * 10) and (trigger_kline[-1] < 10 ) and (trigger_kline[-1] > 7 ) 
    
    drop_from_high_gate = trigger_kline[2]/trigger_kline[4] < 1.05
    trigger_gate = trigger_kline[-1] > 10 
    

    if trigger_gate and drop_from_high_gate:
        print(trigger_kline)
        is_candidate = True
    else:
        is_candidate = False


    return is_candidate

In [13]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[4] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [14]:
trade_list = []
for target_ticker in target_ticker_list:
    
    print('Analyzing Hourly Started ')
    print(target_ticker)

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_1HOUR, "30 day ago UTC") 
    length = int(len(klines))
    if(length < 24):
        continue
    
    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)

    for x in range(0, length):
        
        hit_ticker_dictionary = {}
        
        filtered_klines_info = focused_kline_array[x:x+24]
        
        if(len(filtered_klines_info) == 24):
            context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
            average_volume = np.mean(context_klines_info[:,5])
            tolerable_volume = average_volume
        
        
            buy_kline_info = filtered_klines_info[-1]


            if(is_candidate(filtered_klines_info, tolerable_volume)):

                hit_ticker_dictionary = {}
                buy_price = buy_kline_info[4]
                hit_ticker_dictionary['ticker'] = target_ticker
                hit_ticker_dictionary['buy_price'] = buy_kline_info[4]

                buy_timesstamp = buy_kline_info[0]
                hit_ticker_dictionary['buy_datetime'] = str(datetime.fromtimestamp(buy_timesstamp // 1000))
                
                proft_percentage = 1.10
                loss_percentage = 0.75
                profit_sell_price = buy_price * proft_percentage
                loss_sell_price = buy_price * loss_percentage
                
                
                
                usdt = 1000
                
                buy_quantity = usdt/buy_price
                hit_ticker_dictionary['quantity'] = buy_quantity
                
                for y in range(x+25, length):
                    high_price = focused_kline_array[y-1:y][0][2]
                    low_price = focused_kline_array[y-1:y][0][3]
                    close_price = focused_kline_array[y-1:y][0][4]
                    sell_timesstamp = focused_kline_array[y-1:y][0][0]
                    sell_datetime = str(datetime.fromtimestamp(sell_timesstamp // 1000))


                    if high_price >= profit_sell_price:
                        hit_ticker_dictionary['sell_price'] = profit_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = profit_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * proft_percentage - usdt
                        
                        break
                        
                    elif low_price <= loss_sell_price and False:
                        hit_ticker_dictionary['sell_price'] = loss_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = loss_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * loss_percentage - usdt
                        break
                    elif sell_timesstamp - buy_timesstamp >= 14400000 :
                        hit_ticker_dictionary['sell_price'] = close_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = close_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = buy_quantity * close_price - buy_quantity * buy_price
                        break
                        
                        
                        
                        
                        
                trade_list.append(hit_ticker_dictionary)
                
                busd = 100

                quantity = busd/ buy_kline_info[4]
                quantity_str = "{:0.0{}f}".format(quantity, 2)

                """
                order = client.create_test_order(symbol=target_ticker, side=SIDE_BUY, type='MARKET', quoteOrderQty=busd)
                time.sleep(5)
                order_id = order['orderId'] 
                '''order_id = client.get_all_orders(symbol=target_ticker)[0]['orderId'] '''

                buy_price = order['fills'][0]['price']
                buy_qty = order['executedQty']
                sell_price = buy_price * 1.03



                while client.get_order(symbol=target_ticker, orderId=order_id)['status'] != 'FILLED':
                    time.sleep(10)

                if client.get_order(symbol=target_ticker, orderId=order_id)['status'] == 'FILLED':
                    asset = target_ticker.split("USDT")[0]
                    balance = client.get_asset_balance(asset=asset)
                    free_to_trade = balance['free']
                    sell_price_str = "{:0.0{}f}".format(sell_price, 2)

                    order = client.create_test_order(symbol=target_ticker, side=SIDE_SELL, type='LIMIT', quantity=float(balance['free']), price=sell_price_str, timeInForce=TIME_IN_FORCE_GTC)
                    print('********** SELL ORDER SET ************************')

                    """


Analyzing Hourly Started 
FETUSDT
Analyzing Hourly Started 
MBLUSDT
[1.65674160e+12 2.45700000e-03 2.92900000e-03 2.45000000e-03
 2.86800000e-03 2.76595756e+08 1.67277167e+01]
Analyzing Hourly Started 
STPTUSDT
Analyzing Hourly Started 
ANTUSDT
Analyzing Hourly Started 
SUNUSDT
Analyzing Hourly Started 
UTKUSDT
Analyzing Hourly Started 
XVSUSDT
Analyzing Hourly Started 
ALPHAUSDT
Analyzing Hourly Started 
CTKUSDT
Analyzing Hourly Started 
REEFUSDT
Analyzing Hourly Started 
ALICEUSDT
Analyzing Hourly Started 
PERPUSDT
[1.65585960e+12 5.85000000e-01 6.83000000e-01 5.80000000e-01
 6.51000000e-01 1.13573728e+06 1.12820513e+01]
[1.65588480e+12 6.57000000e-01 8.10000000e-01 6.47000000e-01
 7.76000000e-01 3.69928726e+06 1.81126332e+01]
[1.65589560e+12 7.04000000e-01 7.86000000e-01 7.04000000e-01
 7.80000000e-01 2.21987308e+06 1.07954545e+01]
[1.65803400e+12 7.99000000e-01 8.89000000e-01 7.99000000e-01
 8.86000000e-01 9.77066920e+05 1.08886108e+01]
Analyzing Hourly Started 
TLMUSDT
[1.65713760

In [15]:
trade_list

[{'ticker': 'MBLUSDT',
  'buy_price': 0.002868,
  'buy_datetime': '2022-07-02 08:00:00',
  'quantity': 348675.0348675035,
  'sell_price': 0.0031548,
  'sell_datetime': '2022-07-02 09:00:00',
  'days_to_gain_loss': 0,
  'price_diff': 0.00028680000000000025,
  'proft_loss': 100.0},
 {'ticker': 'PERPUSDT',
  'buy_price': 0.651,
  'buy_datetime': '2022-06-22 03:00:00',
  'quantity': 1536.0983102918585,
  'sell_price': 0.652,
  'sell_datetime': '2022-06-22 07:00:00',
  'days_to_gain_loss': 0,
  'price_diff': 0.0010000000000000009,
  'proft_loss': 1.5360983102918908},
 {'ticker': 'PERPUSDT',
  'buy_price': 0.776,
  'buy_datetime': '2022-06-22 10:00:00',
  'quantity': 1288.659793814433,
  'sell_price': 0.795,
  'sell_datetime': '2022-06-22 14:00:00',
  'days_to_gain_loss': 0,
  'price_diff': 0.019000000000000017,
  'proft_loss': 24.48453608247428},
 {'ticker': 'PERPUSDT',
  'buy_price': 0.78,
  'buy_datetime': '2022-06-22 13:00:00',
  'quantity': 1282.051282051282,
  'sell_price': 0.739,
  's

In [16]:
len(trade_list)

8

In [17]:
df = pd.DataFrame.from_dict(trade_list)

In [18]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] > 0])

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
0,MBLUSDT,0.002868,2022-07-02 08:00:00,348675.034868,0.003155,2022-07-02 09:00:00,0,0.000287,100.000000
1,PERPUSDT,0.651000,2022-06-22 03:00:00,1536.098310,0.652000,2022-06-22 07:00:00,0,0.001000,1.536098
2,PERPUSDT,0.776000,2022-06-22 10:00:00,1288.659794,0.795000,2022-06-22 14:00:00,0,0.019000,24.484536
4,PERPUSDT,0.886000,2022-07-17 07:00:00,1128.668172,0.974600,2022-07-17 10:00:00,0,0.088600,100.000000
5,TLMUSDT,0.027500,2022-07-06 22:00:00,36363.636364,0.030250,2022-07-06 23:00:00,0,0.002750,100.000000
7,POLSUSDT,0.574000,2022-06-29 13:00:00,1742.160279,0.631400,2022-06-29 14:00:00,0,0.057400,100.000000


In [19]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] < 0])

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
3,PERPUSDT,0.780,2022-06-22 13:00:00,1282.051282,0.739,2022-06-22 17:00:00,0,-0.041,-52.564103
6,POLSUSDT,0.526,2022-06-24 09:00:00,1901.140684,0.504,2022-06-24 13:00:00,0,-0.022,-41.825095


In [20]:
with pd.option_context('display.max_rows', None):
    display(df[pd.isna(df['sell_price'])])

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss


In [21]:
df['proft_loss'].sum()

331.6314367716293

In [22]:
with pd.option_context('display.max_rows', None):
    display(df[np.invert(pd.isna(df['sell_price']))])

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
0,MBLUSDT,0.002868,2022-07-02 08:00:00,348675.034868,0.003155,2022-07-02 09:00:00,0,0.000287,100.000000
1,PERPUSDT,0.651000,2022-06-22 03:00:00,1536.098310,0.652000,2022-06-22 07:00:00,0,0.001000,1.536098
2,PERPUSDT,0.776000,2022-06-22 10:00:00,1288.659794,0.795000,2022-06-22 14:00:00,0,0.019000,24.484536
3,PERPUSDT,0.780000,2022-06-22 13:00:00,1282.051282,0.739000,2022-06-22 17:00:00,0,-0.041000,-52.564103
4,PERPUSDT,0.886000,2022-07-17 07:00:00,1128.668172,0.974600,2022-07-17 10:00:00,0,0.088600,100.000000
5,TLMUSDT,0.027500,2022-07-06 22:00:00,36363.636364,0.030250,2022-07-06 23:00:00,0,0.002750,100.000000
6,POLSUSDT,0.526000,2022-06-24 09:00:00,1901.140684,0.504000,2022-06-24 13:00:00,0,-0.022000,-41.825095
7,POLSUSDT,0.574000,2022-06-29 13:00:00,1742.160279,0.631400,2022-06-29 14:00:00,0,0.057400,100.000000
